In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import hdbscan
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("train.csv")
data.head()
testData = pd.read_csv("test.csv")
testData.head()

,ID,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n4086,n4087,n4088,n4089,n4090,n4091,n4092,n4093,n4094,n4095
0,0,0.0,0.000000,0.908889,0.251257,0.662262,0.042495,0.000000,0,0.964784,...,0.694072,1.146161,1.483842,0.717836,0.472616,0.000000,0.488022,0.0,0.0,0.655670
1,1,0.0,0.000000,1.191055,0.407350,0.441898,0.000000,0.334858,0,0.295357,...,0.273436,1.466932,0.940850,0.470344,1.032085,0.000000,0.654070,0.0,0.0,0.614493
2,2,0.0,0.261903,0.992782,0.301102,0.636006,0.009558,0.009448,0,0.974949,...,0.000000,0.769983,0.834360,0.369656,1.000858,0.431571,0.361993,0.0,0.0,0.392158
3,3,0.0,0.000000,1.352401,0.346003,0.401412,0.000000,0.000000,0,0.450667,...,0.339935,1.325595,0.981124,0.486731,0.747392,0.000000,0.300671,0.0,0.0,0.628365
4,4,0.0,0.000000,1.114281,0.696140,0.121505,0.000000,0.000000,0,0.591384,...,0.093661,0.875113,0.360689,0.659230,0.546044,0.000000,0.427255,0.0,0.0,0.835671


In [3]:
x = data.iloc[:, 1:-1].values
xTest = testData.iloc[:, 1:].values
xFeatures = np.array(x, dtype=float)
xTest = np.array(xTest, dtype=float)
xLabels = data.iloc[:, -1].values
xLabels = np.array(xLabels)

In [6]:
scaler = StandardScaler()
pca = PCA(n_components=100)
p1 = pca.fit(scaler.fit_transform(xFeatures))
x_processed = p1.transform(xFeatures)
x_test_processed = p1.transform(xTest)
print(x_processed.shape, x_test_processed.shape)

(1216, 100) (415, 100)


In [13]:
isolation_forest = IsolationForest(n_estimators=500, contamination=0.1)
isolation_forest.fit(x_processed)
indices = np.where(isolation_forest.predict(x_processed) != -1)[0]
x_transformed = x_processed[indices]
y_transformed = xLabels[indices]

#clustering
clusterer1 = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
cluster_labels1 = clusterer1.fit_predict(x_transformed)
x_clustered = np.concatenate((x_transformed, cluster_labels1.reshape(-1, 1)), axis=1)

clusterer2 = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
cluster_labels2 = clusterer2.fit_predict(x_test_processed)
x_test_clustered = np.concatenate((x_test_processed, cluster_labels2.reshape(-1, 1)), axis=1)

# x_train, x_validate, y_train, y_validate = train_test_split(x_clustered, y_transformed, test_size=0.3, random_state=1)
# print(set(x_test_clustered[:, -1]))
#classification
rf = RandomForestClassifier(n_estimators=1000, max_depth=17)

In [14]:
print(np.mean(cross_val_score(rf, x_clustered, y_transformed, cv=5)))


0.7129655230195635
